In [15]:
from collections import namedtuple


Result = namedtuple('Result', ('nfev', 'cost', 'gradnorm', 'x'))

Result.__doc__ = """Результаты оптимизации



Attributes

----------

nfev : int

    Полное число вызовов модельной функции

cost : 1-d array

    Значения функции потерь 0.5 sum(y - f)^2 на каждом итерационном шаге.

    В случае метода Гаусса—Ньютона длина массива равна nfev, в случае ЛМ-метода

    длина массива — менее nfev

gradnorm : float

    Норма градиента на финальном итерационном шаге

x : 1-d array

    Финальное значение вектора, минимизирующего функцию потерь

"""

In [45]:
import numpy as np

In [99]:
def gauss_newton(y, f, j, x0, k=1, tol=1e-4):
    nfev = 0
    cost = []
    x = x0.copy()
    
    while True:
        nfev +=1
        residual = y - f(*x)
        cost.append(0.5 * residual @ residual )
        jac = j(*x)
        grad = jac.T @ residual
        gradnorm  = np.linalg.norm(grad)
        delta_x = np.linalg.solve(jac.T @ jac, grad)
        if np.linalg.norm(delta_x) <= tol:
            break
        x += k * delta_x
    cost  = np.array(cost)
    return Result(nfev, cost, gradnorm, x)
        
        

In [89]:
def f(t, a, b, c):
    return b * np.exp(-a * t) * t**2 + c

def j(t, a, b, c):
    j  = np.empty((t.size, 3))
    j[:, 0] = -b * np.exp(-a * t) * t**3
    j[:, 1] = np.exp(-a * t) * t**2
    j[:, 2] = 1 
    return j

_a = 0.75
_b = 2
_c = 0.5
_n = 30
_x = (_a, _b, _c)



In [92]:
%%timeit
t = np.linspace(0, 10, _n)
y = f(t, *_x) + np.random.normal(0, 0.1, _n)

r = gauss_newton(y = y, 
                 f = lambda *args: f(t, *args), 
                 j = lambda *args: j(t, *args), 
                 x0 = np.array([1.0, 1.0, 1.0]),
                k = 0.04, tol=1e-3)


8.52 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [105]:

#%%timeit
t = np.random.normal(5, 4, _n)
y = f(t, *_x) + np.random.normal(0, 0.1, _n)

r = gauss_newton(y = y, 
                 f = lambda *args: f(t, *args), 
                 j = lambda *args: j(t, *args), 
                 x0 = np.array([1.0, 1.0, 1.0]),
                k = 0.04, tol=1e-3)
print(r.cost[-1])

0.08455084847496208


In [112]:
t = np.linspace(0, 10, _n)
y = f(t, *_x) + np.random.normal(0, 0.1, _n)

r = gauss_newton(y = y, 
                 f = lambda *args: f(t, *args), 
                 j = lambda *args: j(t, *args), 
                 x0 = np.array([1.0, 1.0, 1.0]),
                k = 0.04, tol=1e-3)
print(r.cost[-1])

0.12019119656613689


In [101]:
print(r.cost[-1])

0.19241663671306364
